<a href="https://colab.research.google.com/github/Nikhileswar-Komati/Suicide_Ideation/blob/master/Logistic_Regression_74.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Install java
! apt-get update -qq
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version
# Install pyspark
! pip install --ignore-installed pyspark==2.4.4
# Install Spark NLP
! pip install --ignore-installed spark-nlp

openjdk version "1.8.0_275"
OpenJDK Runtime Environment (build 1.8.0_275-8u275-b01-0ubuntu1~18.04-b01)
OpenJDK 64-Bit Server VM (build 25.275-b01, mixed mode)
Processing /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471/pyspark-2.4.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e3/53/c737818eb9a7dc32a7cd4f1396e787bd94200c3997c72c1dbe028587bd76/py4j-0.10.7-py2.py3-none-any.whl
Processing /root/.cache/pip/wheels/75/e4/63/ade6bdbafb87b93e87b21c22db80b606eb4d47d7aaa88e8158/spark_nlp-2.7.0-py2.py3-none-any.whl


In [2]:
import sparknlp
spark = sparknlp.start()
sparknlp.version()
spark.version

'2.4.4'

In [13]:
from pyspark.ml import Pipeline
from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *
from pyspark.ml.feature import *
from pyspark.ml.classification import *

In [4]:
import os
#kaggle username from api token 
os.environ['KAGGLE_USERNAME'] = "nikhileswarkomati"
#kaggle key from api token
os.environ['KAGGLE_KEY'] ="0ffd0be5c369d70815be7d0521aeab91"
#dataset path for downloading the data
!kaggle datasets download -d nikhileswarkomati/suicide-watch

suicide-watch.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
! unzip '/content/suicide-watch.zip'

Archive:  /content/suicide-watch.zip
  inflating: Cleaned_Depression_Vs_Suicide.csv  
  inflating: pushshift_SW_2L.csv     
  inflating: pushshift_dep_2L.csv    


In [5]:
spark_df = spark.read.option("header", "true").option("multiLine", "true").option("quote", "\"").option("escape", "\"").option("inferSchema", "true").csv('/content/Cleaned_Depression_Vs_Suicide.csv', sep = ',')

In [6]:
spark_df.show()

+-------+--------------------+--------------------+-----+-------------------+--------------------+-------+------------+
|Post_id|               Title|                Body|Score|       Publish_date|Total_no_of_comments|Over_18|   Subreddit|
+-------+--------------------+--------------------+-----+-------------------+--------------------+-------+------------+
| 2nfysn|How do I know if ...|I have pretty sev...|    4|2014-11-26 03:33:59|                   3|  false|  depression|
|  pn3br|I don't know if t...|Emotions are a on...|    8|2012-02-13 05:26:20|                   2|  false|  depression|
| kjpeqe|The best christma...|All I want for ch...|    1|2020-12-24 23:54:44|                   3|  false|SuicideWatch|
| 4bf9zo|People only talk ...|Whenever I talk t...|   10|2016-03-22 01:03:22|                   1|  false|  depression|
| gxl5kl|What to do before...|So I plan on kill...|    1|2020-06-06 05:30:53|                   6|  false|SuicideWatch|
| dgdwh4|   It needs to stop!|If you cou

In [7]:
spark_split_df = spark_df

(train_data, test_data, val_data) = spark_split_df.randomSplit([0.1, 0.3, 0.3], 24)

print("Train length", train_data.count())
print("Test length", test_data.count())
print("validation length", val_data.count())

Train length 91801
Test length 275056


In [9]:
document = DocumentAssembler()\
  .setInputCol("Body")\
  .setOutputCol("document")
sentence = SentenceDetector()\
  .setInputCols(['document'])\
  .setOutputCol('sentence')
token = Tokenizer()\
  .setInputCols(['sentence'])\
  .setOutputCol('token')
stop_words = StopWordsCleaner.pretrained('stopwords_tr', 'tr')\
  .setInputCols(["token"]) \
  .setOutputCol("cleanTokens") \
  .setCaseSensitive(False)
lemmatizer = LemmatizerModel.pretrained("lemma", "tr") \
  .setInputCols(["cleanTokens"]) \
  .setOutputCol("lemma")
finisher = Finisher() \
  .setInputCols(["lemma"]) \
  .setOutputCols(["token_features"]) \
  .setOutputAsArray(True) \
  .setCleanAnnotations(False)


stopwords_tr download started this may take some time.
Approximate size to download 2 KB
[OK!]
lemma download started this may take some time.
Approximate size to download 14.8 MB
[OK!]


In [14]:

hashTF = HashingTF(inputCol="token_features", outputCol="raw_features")
idf = IDF(inputCol="raw_features", outputCol="features", minDocFreq=5)
label_strIdx = StringIndexer(inputCol="Subreddit", outputCol="label")
logReg = LogisticRegression(maxIter=10)
label_Idxstr = IndexToString(inputCol="label", outputCol="article_class")
# Pipeline for Logistic Regression Classifier
nlp_pipeline_lr = Pipeline(
    stages=[document,
    sentence,
    token,
    stop_words,
    lemmatizer,
    finisher,
    hashTF,
    idf,
    label_strIdx,
    logReg,
    label_Idxstr])

In [15]:
classification_model_lr = nlp_pipeline_lr.fit(train_data)

In [16]:
from sklearn.metrics import classification_report, accuracy_score
# Converting Spark DF into Pandas DF
df_lr = classification_model_lr.transform(test_data).select("Subreddit", "label", "prediction").toPandas()
print(classification_report(df_lr.label, df_lr.prediction))

              precision    recall  f1-score   support

         0.0       0.75      0.77      0.76    144293
         1.0       0.74      0.71      0.73    130763

    accuracy                           0.74    275056
   macro avg       0.74      0.74      0.74    275056
weighted avg       0.74      0.74      0.74    275056



In [17]:
print(accuracy_score(df_lr.label, df_lr.prediction))

0.7448483217962888
